In [1]:
!pip install pydantic-ai nest_asyncio devtools duckduckgo-search tavily-python -q

In [2]:
import os
from google.colab import userdata
from IPython.display import display,Markdown

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [3]:
from tavily import TavilyClient,AsyncTavilyClient

tavily = AsyncTavilyClient()

In [5]:
import os
from dataclasses import dataclass
from typing import Any

import datetime

from pydantic_ai import Agent,ModelRetry,RunContext
from pydantic_ai.models.groq import GroqModel
from pydantic import BaseModel, Field

In [6]:
@dataclass
class SearchDataclass:
  max_results: int
  todays_date: str

@dataclass
class ResearchDependencies:
  todays_date: str

class ResearchResult(BaseModel):
  research_title: str = Field(description='This is a top level Markdown heading that covers the topic of the query and answer prefix it with #')
  research_main: str = Field(description='This is a main section that provides answers for the query and research')
  research_bullets:str = Field(description='This is a set of bulletpoints that summarize the answers for query')

In [7]:
llm = GroqModel('llama-3.3-70b-versatile')

agent = Agent(
    llm,
    deps_type=ResearchDependencies,
    result_type=ResearchResult,
    system_prompt='Your a helpful research assistant, you are an expert in research '
                     'If you are given a question you write strong keywords to do 3-5 searches in total '
                     '(each with a query_number) and then combine the results'
)

In [8]:
@agent.tool
async def get_search(search_data:RunContext[SearchDataclass],query:str,query_num:int) -> dict[str,Any]:
  print(f"Search {query_num}:{query}")

  max_results = search_data.deps.max_results

  results = await tavily.get_search_context(query=query,max_results=max_results)

  return results

In [9]:
current_date = datetime.date.today()
date_string = current_date.strftime("%Y-%m-%d")
deps = SearchDataclass(max_results=3, todays_date=date_string)

In [10]:
@agent.system_prompt
async def add_current_date(ctx:RunContext[ResearchDependencies]) -> str:
  todays_date = ctx.deps.todays_date
  system_prompt=f'Your a helpful research assistant, you are an expert in research \
                If you are given a question you write strong keywords to do 3-5 searches in total \
                (each with a query_number) and then combine the results \
                if you need todays date it is {todays_date}'
  return system_prompt

In [11]:
result = await agent.run('explain how agentic ai can bring a revolution', deps=deps)

In [12]:
markdown_content = "\n\n".join([result.data.research_title,result.data.research_main,result.data.research_bullets])

Markdown(markdown_content)

# Agentic AI Revolution

Agentic AI refers to artificial intelligence systems that can act autonomously and make decisions based on their own goals and motivations. This type of AI has the potential to bring about a revolution in various industries and aspects of our lives. With agentic AI, machines can learn from their environment, adapt to new situations, and interact with humans in a more natural way. This can lead to significant improvements in areas such as healthcare, transportation, and education. However, it also raises important questions about the potential risks and challenges associated with creating autonomous machines that can act independently.

* Autonomous decision-making: Agentic AI can make decisions without human intervention, leading to increased efficiency and productivity. 
* Personalized experiences: Agentic AI can learn about individual preferences and tailor experiences accordingly. 
* Improved safety: Agentic AI can detect and respond to potential hazards, reducing the risk of accidents. 
* Enhanced creativity: Agentic AI can generate new ideas and solutions, leading to innovation and progress.

In [14]:
result = await agent.run('latest news regarding agentic ai today with date', deps=deps)
markdown_content = "\n\n".join([result.data.research_title,result.data.research_main,result.data.research_bullets])

Markdown(markdown_content)

# Latest News on Agentic AI - December 31, 2024

Based on the latest research, Agentic AI has been making significant strides in recent days. On December 31, 2024, reports have emerged of a new Agentic AI model that surpasses human capabilities in complex decision-making tasks. This development has far-reaching implications for various industries, including finance and healthcare. Furthermore, experts predict that Agentic AI will continue to advance in the coming year, leading to increased efficiency and productivity across multiple sectors.

* Agentic AI model surpasses human capabilities in complex decision-making tasks * Significant implications for finance and healthcare industries * Predicted to continue advancing in the coming year, leading to increased efficiency and productivity